In [261]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [262]:
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

In [263]:
#importing Lewis & Clark County Water Data
mgmb = pd.read_excel('MBMG_SWL_09072022.xlsx') 

#importing wells
wells = pd.read_csv('Wells.csv', encoding='windows-1252', low_memory=False)

In [264]:
#preparing to merge dataframes
wells = wells.drop(['geomethod','datum','township','range','section','quarter_section','type','date_completed'], axis=1)

#merging two data sets
final = pd.concat([wells, mgmb], axis=0)

In [265]:
#removing null of swl_ground 
final = final.dropna(subset=['swl_ground'])

#converting to datetime
final['date_measured'] = pd.to_datetime(final['date_measured']).dt.strftime('%m-%d-%Y')

#removing duplicates of gwicid, date_measured, swl_ground and keeping the last (transducer data)
final = final.drop_duplicates(subset=['gwicid','date_measured', 'swl_ground'], keep='last')

In [266]:
#observation statistics
a = len(final)
well = final['gwicid'].nunique()
avg = round(a/well,2)
print(f'{a} observations for {well} wells with an average of {avg} per well')

416199 observations for 632 wells with an average of 658.54 per well


In [267]:
#adding months to bin into seasons
final['date_measured'] = pd.to_datetime(final['date_measured'])
final['month'] = final['date_measured'].map(lambda x: x.month)
final['year'] = final['date_measured'].map(lambda x: x.year)
final['season'] = final['month'].copy()

#categorizing by season
bins = [0, 3, 6, 9, 12]
labels = ['winter', 'spring','summer','fall' ]
final['season'] =pd.cut(final['season'], bins=bins, labels=labels)

In [268]:
#binning years
bins = [0, 1999, 2003, 2007, 2011, 2015, 2019, 2023]
labels = ['prior to 2000','2000-2003','2004-2007','2008-2011','2012-2015','2016-2019','2020-2023']
final['time_period'] = pd.cut(final['year'], bins=bins, labels=labels)

In [269]:
#df groupby object of earliest dates
earliest =  final.groupby(['gwicid'])[['date_measured']].min().reset_index()
earliest = earliest.rename(columns= {'date_measured':'latest'})

#df groupby object of latest dates
latest = final.groupby(['gwicid'])[['date_measured']].max().reset_index()
latest = latest.rename(columns={'date_measured':'earliest'})

#mereging groupby objects
time = earliest.merge(latest, on='gwicid')
time['latest'] = pd.to_datetime(time['latest'])
time['earliest'] = pd.to_datetime(time['earliest'])

#calculating length of study
time['length_of_study'] = np.abs(time['earliest'] - time['latest'])
time['diff_in_years'] = round(time['length_of_study'] / timedelta(365),2)

#merging with final dataframe
final = final.merge(time, how='left', on='gwicid')

final.shape

(416199, 19)

In [270]:
#filtering final dataframe to count observations after to 2000
timeframe = '2000-01-01'
obs_filtered = final[(final['date_measured'] > timeframe)]

#list of gwics meeting duration requirement in years
duration_req = 10
gwics_duration_req = obs_filtered[obs_filtered['diff_in_years'] >= duration_req]
duration_req = gwics_duration_req['gwicid'].unique()

#number of gwics meeting length of study requirement
num_gwics_duration_req = len(duration_req)

In [271]:
#list of gwics meeting the average yearly requirement of x observations/year
min_obs_year = 8
yearly_req = obs_filtered.groupby(['gwicid','year']).agg('count')['site_name'].reset_index()
yearly_req = yearly_req.groupby(['gwicid']).agg('mean')['site_name'].reset_index()
yearly_req = yearly_req[yearly_req['site_name'] >= min_obs_year]
yearly_req = yearly_req['gwicid'].unique()

#number of gwics meeting yearly requirement 
num_gwics_yearly_req = len(yearly_req)

In [272]:
#list of gwics meeting the average per year per seasonal requirement
per_seas_per_year_avg = 1.75
season_req = obs_filtered.groupby(['gwicid', 'year','season']).agg('count')['site_name'].reset_index()
season_req = season_req.groupby(['gwicid']).agg('mean')['site_name'].reset_index()
season_req = season_req[season_req['site_name'] >= per_seas_per_year_avg]
season_req = season_req['gwicid'].unique()

#number of gwics the average per year per seasonal requirement
num_per_seas_per_year_avg = len(season_req)

In [273]:
#filtering the final dataframe by gwics meeting both the duration, yearly, and per year per season
final = final[(final['gwicid'].isin(duration_req)) & (final['gwicid'].isin(yearly_req)) &
             (final['gwicid'].isin(season_req))]

#final samples number
sample_number = final['gwicid'].nunique()

print(f'{num_gwics_duration_req} wells meet the duration requirement.')
print(f'{num_gwics_yearly_req} wells meet the minimum  average yearly observations requirement.')
print(f'{num_per_seas_per_year_avg} wells meet the minimum average per year per season requirement.')
print(f'{sample_number} wells meet all requirements.')

167 wells meet the duration requirement.
202 wells meet the minimum  average yearly observations requirement.
123 wells meet the minimum average per year per season requirement.
67 wells meet all requirements.


In [274]:
final['gwicid'].nunique()

67

In [275]:
#analysis of each gwics average swl_ground per time_period per season 
final1 = final[['gwicid','time_period','season','swl_ground']]
final1 = final1.groupby(['gwicid','time_period','season'])[['swl_ground']].mean().reset_index()

#calculating average swl_ground water % change per time period per season 
final1['change'] = final1.groupby(['gwicid','season'])['swl_ground'].pct_change()
final1['change'] = round(final1['change'],2)

final1

,gwicid,time_period,season,swl_ground,change
0,5756,prior to 2000,winter,23.087500,NaN
1,5756,prior to 2000,spring,23.336000,NaN
2,5756,prior to 2000,summer,16.832000,NaN
3,5756,prior to 2000,fall,22.020000,NaN
4,5756,2000-2003,winter,30.560000,0.32
...,...,...,...,...,...
1871,892195,2016-2019,fall,8.750000,-0.09
1872,892195,2020-2023,winter,10.554286,0.04
1873,892195,2020-2023,spring,9.975714,0.19
1874,892195,2020-2023,summer,10.125000,0.01


In [276]:
#calculating count of observations per gwic, per time_cat, per season
final2 = final[['gwicid','time_period','season','swl_ground']]
final2 = final2.groupby(['gwicid','time_period','season'])[['swl_ground']].count().reset_index()
final2 = final2.rename(columns={'swl_ground':'count'})

final2

,gwicid,time_period,season,count
0,5756,prior to 2000,winter,8
1,5756,prior to 2000,spring,10
2,5756,prior to 2000,summer,5
3,5756,prior to 2000,fall,2
4,5756,2000-2003,winter,6
...,...,...,...,...
1871,892195,2016-2019,fall,10
1872,892195,2020-2023,winter,7
1873,892195,2020-2023,spring,14
1874,892195,2020-2023,summer,6


In [277]:
#merging final analysis dataframes for visualization
Final = final2.merge(final1, how='left', on=['gwicid', 'time_period','season'])

# prepating to merge with final to get lat-long data
lat_long = final[['gwicid','latitude', 'longitude','year']]
lat_long = final.groupby(['gwicid','latitude','longitude'])['year'].mean().reset_index()

Final

,gwicid,time_period,season,count,swl_ground,change
0,5756,prior to 2000,winter,8,23.087500,NaN
1,5756,prior to 2000,spring,10,23.336000,NaN
2,5756,prior to 2000,summer,5,16.832000,NaN
3,5756,prior to 2000,fall,2,22.020000,NaN
4,5756,2000-2003,winter,6,30.560000,0.32
...,...,...,...,...,...,...
1871,892195,2016-2019,fall,10,8.750000,-0.09
1872,892195,2020-2023,winter,7,10.554286,0.04
1873,892195,2020-2023,spring,14,9.975714,0.19
1874,892195,2020-2023,summer,6,10.125000,0.01


In [278]:
Final = pd.merge(Final, lat_long, how='left', on ='gwicid')
Final = Final.drop(labels = 'year', axis=1)
Final

,gwicid,time_period,season,count,swl_ground,change,latitude,longitude
0,5756,prior to 2000,winter,8,23.087500,NaN,46.643954,-112.046558
1,5756,prior to 2000,spring,10,23.336000,NaN,46.643954,-112.046558
2,5756,prior to 2000,summer,5,16.832000,NaN,46.643954,-112.046558
3,5756,prior to 2000,fall,2,22.020000,NaN,46.643954,-112.046558
4,5756,2000-2003,winter,6,30.560000,0.32,46.643954,-112.046558
...,...,...,...,...,...,...,...,...
1871,892195,2016-2019,fall,10,8.750000,-0.09,46.645865,-112.015911
1872,892195,2020-2023,winter,7,10.554286,0.04,46.645865,-112.015911
1873,892195,2020-2023,spring,14,9.975714,0.19,46.645865,-112.015911
1874,892195,2020-2023,summer,6,10.125000,0.01,46.645865,-112.015911


In [279]:
Final['gwicid'].nunique()

67

In [280]:
Final.to_csv('swl_analysis.csv')

In [ ]:
#data cleaning (na's and formatting)
Final = Final[Final['count'] != 0]

#filling na change with 0 to indicate no change (or first observation where change is inapplicable)
Final['change'] = Final['change'].fillna(0)

#eliminating 'prior to 2000' time category
Final = Final[Final['time_category'] != 'prior to 2000']
Final
